In [5]:
import json
import numpy as np
from torchvision import datasets, transforms
from kafka import KafkaProducer, KafkaConsumer
import redis
import time
import threading

# Redis setup
redis_client = redis.Redis(host='host.docker.internal', port=6379, db=0)

# Activation functions
def relu(x):
    return np.maximum(0, x)

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

ACTIVATIONS = {
    "relu": relu,
    "softmax": softmax
}

class Neuron(threading.Thread):
    def __init__(self, layer_id, neuron_id, weights, bias, activation, is_final_layer=False):
        threading.Thread.__init__(self)
        self.layer_id = layer_id
        self.neuron_id = neuron_id
        self.weights = np.array(weights)
        self.bias = np.array(bias)
        self.activation_func = None if is_final_layer else ACTIVATIONS.get(activation, relu)
        self.is_final_layer = is_final_layer
        self.output = None

        # Initialize Kafka Consumer
        self.consumer = KafkaConsumer(
            f'layer-{self.layer_id[-1]}',
            bootstrap_servers='kafka:9092',
            value_deserializer=lambda m: json.loads(m.decode('utf-8')),
            auto_offset_reset='latest',
            enable_auto_commit=True,
            consumer_timeout_ms=60000
        )

    def run(self):
        print(f"🕒 Neuron {self.neuron_id} in {self.layer_id} waiting for data...")

        for message in self.consumer:
            if message.value.get('layer') == self.layer_id and message.value.get('data') is not None:
                print(f"✅ Neuron {self.neuron_id} in {self.layer_id} received data and started processing...")
                inputs = np.array(message.value['data'])
                self.output = self.process_data(inputs)
                break

        self.consumer.close()

    def process_data(self, inputs):
        z = np.dot(inputs, self.weights) + self.bias
        return z if self.is_final_layer else self.activation_func(z)

class Layer:
    def __init__(self, layer_id, neuron_configs, is_final_layer=False):
        self.layer_id = layer_id
        self.neuron_configs = neuron_configs  # Save neuron configurations
        self.is_final_layer = is_final_layer
        self.neurons = []

    def initialize_neurons(self):
        self.neurons = [
            Neuron(
                layer_id=self.layer_id,
                neuron_id=idx,
                weights=neuron['weights'],
                bias=neuron['biases'],
                activation=neuron['activation'],
                is_final_layer=self.is_final_layer
            )
            for idx, neuron in enumerate(self.neuron_configs)
        ]

    def forward(self, input_data):
        self.initialize_neurons()

        # Start neuron threads
        for neuron in self.neurons:
            neuron.start()

        time.sleep(2)

        # Send data to neurons
        producer = KafkaProducer(
            bootstrap_servers='kafka:9092',
            value_serializer=lambda v: json.dumps(v).encode('utf-8')
        )
        data_message = {'layer': self.layer_id, 'data': input_data.tolist()}

        for neuron_id in range(len(self.neurons)):
            producer.send(f'layer-{self.layer_id[-1]}', key=str(neuron_id).encode(), value=data_message)
            print(f"✅ Layer {self.layer_id} sent data to Neuron {neuron_id}")

        producer.flush()
        producer.close()

        # Wait for neurons to process
        for neuron in self.neurons:
            neuron.join()

        outputs = np.array([neuron.output for neuron in self.neurons])

        if self.is_final_layer:
            outputs = softmax(outputs)

        redis_client.set(self.layer_id, outputs.astype(np.float32).tobytes())

        if not self.is_final_layer:
            self.activate_next_layer()

        return outputs

    def activate_next_layer(self):
        producer = KafkaProducer(bootstrap_servers='kafka:9092',
                                 value_serializer=lambda v: json.dumps(v).encode('utf-8'))
        next_layer = f'layer-{int(self.layer_id[-1]) + 1}'
        print(f"🚀 Activating next layer: {next_layer}")
        producer.send('activate-layer', {'layer': next_layer})
        producer.flush()
        producer.close()

def load_network(filename):
    with open(filename, 'r') as f:
        return json.load(f)

def build_network(json_data):
    layers = []
    sorted_layers = sorted(json_data.keys(), key=lambda x: int(x.split('_')[-1]))
    for i, layer_name in enumerate(sorted_layers):
        layer_info = json_data[layer_name]
        neuron_configs = layer_info['nodes']
        layers.append(Layer(layer_id=layer_name, neuron_configs=neuron_configs, is_final_layer=(i == len(sorted_layers) - 1)))
    return layers

def forward_pass(layers, input_data, image_id):
    for layer in layers:
        input_data = layer.forward(input_data)

    prediction = int(np.argmax(input_data))
    redis_client.hset('predictions', image_id, prediction)
    return prediction

# Load network
data = load_network("node_based_model.json")
network = build_network(data)

# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
mnist_test = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# Test first 10 images one by one
for i in range(10):
    image, label = mnist_test[i]
    image_np = image.view(-1).numpy()
    prediction = forward_pass(network, image_np, i)
    print(f"Image {i} Prediction: {prediction}, Label: {label}")

# Calculate accuracy
predictions = redis_client.hgetall('predictions')
correct = sum(int(predictions[k]) == mnist_test[int(k)][1] for k in predictions)
accuracy = correct / len(predictions)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


🕒 Neuron 0 in layer_0 waiting for data...
🕒 Neuron 1 in layer_0 waiting for data...
🕒 Neuron 2 in layer_0 waiting for data...
🕒 Neuron 3 in layer_0 waiting for data...
🕒 Neuron 4 in layer_0 waiting for data...
🕒 Neuron 5 in layer_0 waiting for data...
🕒 Neuron 6 in layer_0 waiting for data...
🕒 Neuron 7 in layer_0 waiting for data...
🕒 Neuron 8 in layer_0 waiting for data...
🕒 Neuron 9 in layer_0 waiting for data...
🕒 Neuron 10 in layer_0 waiting for data...
🕒 Neuron 11 in layer_0 waiting for data...
🕒 Neuron 12 in layer_0 waiting for data...
🕒 Neuron 13 in layer_0 waiting for data...
🕒 Neuron 14 in layer_0 waiting for data...
🕒 Neuron 15 in layer_0 waiting for data...
🕒 Neuron 16 in layer_0 waiting for data...
🕒 Neuron 17 in layer_0 waiting for data...
🕒 Neuron 18 in layer_0 waiting for data...
🕒 Neuron 19 in layer_0 waiting for data...
🕒 Neuron 20 in layer_0 waiting for data...
🕒 Neuron 21 in layer_0 waiting for data...
🕒 Neuron 22 in layer_0 waiting for data...
🕒 Neuron 23 in layer_